In [1]:
import requests
import json
import pandas as pd
from datetime import datetime, timedelta

# Base URL for historical data
BASE_URL = "https://history.deribit.com/api/v2/public"


->

# Testing Deribit Historical Data Endpoints

## 1. Get Instruments
First, let's fetch the available instruments for a given currency (BTC or ETH)

In [2]:
# Get all BTC instruments
url = f"{BASE_URL}/get_instruments"
params = {
    "currency": "BTC",
    "kind": "future"  # Options: future, option, spot, future_combo, option_combo
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
print(f"Number of instruments: {len(data['result']) if 'result' in data else 0}")
print("\nFirst few instruments:")
if 'result' in data and len(data['result']) > 0:
    for instrument in data['result'][:5]:
        print(f"  - {instrument['instrument_name']}: {instrument.get('kind', 'N/A')}")
else:
    print(json.dumps(data, indent=2))


Status Code: 200
Number of instruments: 8

First few instruments:
  - BTC-24OCT25: future
  - BTC-31OCT25: future
  - BTC-28NOV25: future
  - BTC-26DEC25: future
  - BTC-27MAR26: future


In [3]:
# Get details of a specific instrument
url = f"{BASE_URL}/get_instrument"
params = {
    "instrument_name": "BTC-PERPETUAL"
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
if 'result' in data:
    print(json.dumps(data['result'], indent=2))
else:
    print(json.dumps(data, indent=2))


Status Code: 200
{
  "tick_size_steps": [],
  "tick_size": 0.5,
  "taker_commission": 0.0005,
  "settlement_period": "perpetual",
  "settlement_currency": "BTC",
  "rfq": false,
  "quote_currency": "USD",
  "price_index": "btc_usd",
  "min_trade_amount": 10.0,
  "max_liquidation_commission": 0.0075,
  "max_leverage": 50,
  "maker_commission": 0.0,
  "kind": "future",
  "is_active": true,
  "instrument_name": "BTC-PERPETUAL",
  "instrument_id": 210838,
  "future_type": "reversed",
  "expiration_timestamp": 32503708800000,
  "creation_timestamp": 1534242287000,
  "counter_currency": "USD",
  "contract_size": 10.0,
  "block_trade_tick_size": 0.01,
  "block_trade_min_trade_amount": 100000,
  "block_trade_commission": 0.00025,
  "base_currency": "BTC"
}


## 2. Get Historical Trades by Currency
Fetch historical trades for a given currency


In [4]:
# Get last trades by currency
url = f"{BASE_URL}/get_last_trades_by_currency"
params = {
    "currency": "BTC",
    "count": 10  # Number of trades to retrieve
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
if 'result' in data and 'trades' in data['result']:
    trades = data['result']['trades']
    print(f"Number of trades retrieved: {len(trades)}")
    print("\nFirst few trades:")
    for trade in trades[:3]:
        timestamp = datetime.fromtimestamp(trade['timestamp'] / 1000)
        print(f"  {timestamp} - {trade['instrument_name']}: {trade['amount']} @ {trade['price']}")
else:
    print(json.dumps(data, indent=2))


Status Code: 200
Number of trades retrieved: 10

First few trades:
  2025-10-20 10:19:29.856000 - BTC-28NOV25: 100.0 @ 111505.0
  2025-10-20 10:19:29.160000 - BTC-25SEP26: 16380.0 @ 118125.0
  2025-10-20 10:19:29.113000 - BTC-PERPETUAL: 2480.0 @ 110810.5


## 3. Get Historical Trades by Time Range
Fetch trades for a specific time period


In [5]:
# Get trades for a specific time range
# Timestamps should be in milliseconds
end_time = int(datetime.now().timestamp() * 1000)
start_time = int((datetime.now() - timedelta(days=1)).timestamp() * 1000)

url = f"{BASE_URL}/get_last_trades_by_currency_and_time"
params = {
    "currency": "BTC",
    "start_timestamp": start_time,
    "end_timestamp": end_time,
    "count": 100
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
if 'result' in data and 'trades' in data['result']:
    trades = data['result']['trades']
    print(f"Number of trades retrieved: {len(trades)}")
    print(f"Time range: {datetime.fromtimestamp(start_time/1000)} to {datetime.fromtimestamp(end_time/1000)}")
    
    if len(trades) > 0:
        df = pd.DataFrame(trades)
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        print("\nTrade summary:")
        print(df[['datetime', 'instrument_name', 'price', 'amount', 'direction']].head(10))
else:
    print(json.dumps(data, indent=2))


Status Code: 200
Number of trades retrieved: 100
Time range: 2025-10-19 10:20:03.683000 to 2025-10-20 10:20:03.683000

Trade summary:
                 datetime       instrument_name        price   amount  \
0 2025-10-20 09:20:02.145         BTC-PERPETUAL  110852.5000     10.0   
1 2025-10-20 09:20:02.128         BTC-PERPETUAL  110850.0000     20.0   
2 2025-10-20 09:20:02.125         BTC-PERPETUAL  110846.0000    240.0   
3 2025-10-20 09:20:01.397         BTC-PERPETUAL  110842.5000    170.0   
4 2025-10-20 09:20:01.397         BTC-PERPETUAL  110842.5000     30.0   
5 2025-10-20 09:20:01.397         BTC-PERPETUAL  110842.5000     30.0   
6 2025-10-20 09:20:00.567  BTC-31OCT25-145000-C       0.0007      1.0   
7 2025-10-20 09:19:59.809  BTC-21OCT25-109000-P       0.0031      0.9   
8 2025-10-20 09:19:59.447         BTC-PERPETUAL  110835.0000  21760.0   
9 2025-10-20 09:19:59.447         BTC-PERPETUAL  110835.0000   3720.0   

  direction  
0       buy  
1       buy  
2       buy  
3     

## 4. Get Historical Trades by Specific Instrument
Fetch trades for a specific instrument like BTC-PERPETUAL

In [6]:
# Get trades for a specific instrument
url = f"{BASE_URL}/get_last_trades_by_instrument"
params = {
    "instrument_name": "BTC-PERPETUAL",
    "count": 50
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
if 'result' in data and 'trades' in data['result']:
    trades = data['result']['trades']
    print(f"Number of trades retrieved: {len(trades)}")
    
    if len(trades) > 0:
        df = pd.DataFrame(trades)
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        print("\nBTC-PERPETUAL trades:")
        print(df[['datetime', 'price', 'amount', 'direction']].head(10))
        print(f"\nPrice range: ${df['price'].min():.2f} - ${df['price'].max():.2f}")
        print(f"Total volume: {df['amount'].sum():.2f} BTC")
else:
    print(json.dumps(data, indent=2))


Status Code: 200
Number of trades retrieved: 50

BTC-PERPETUAL trades:
                 datetime     price   amount direction
0 2025-10-20 09:20:22.519  110885.0    200.0       buy
1 2025-10-20 09:20:22.519  110885.0  30930.0       buy
2 2025-10-20 09:20:22.519  110885.0   3790.0       buy
3 2025-10-20 09:20:13.580  110891.0     20.0      sell
4 2025-10-20 09:20:13.571  110891.0     60.0      sell
5 2025-10-20 09:20:13.569  110891.0     60.0      sell
6 2025-10-20 09:20:08.864  110888.0  24290.0       buy
7 2025-10-20 09:20:08.856  110888.0  14230.0       buy
8 2025-10-20 09:20:08.856  110888.0   9900.0       buy
9 2025-10-20 09:20:08.855  110888.0  29180.0       buy

Price range: $110820.00 - $110891.00
Total volume: 468910.00 BTC


## 5. Get Historical Data from 2016 (Deribit Launch)
Test accessing really old data to verify complete historical access


In [45]:
# Get trades from 2016 (Deribit's launch)
# Let's try getting data from early 2017
start_time = int(datetime(2018, 1, 1).timestamp() * 1000)
end_time = int(datetime(2025, 10, 20).timestamp() * 1000)

url = f"{BASE_URL}/get_last_trades_by_instrument_and_time"
params = {
    "instrument_name": "BTC-17OCT25",
    "start_timestamp": start_time,
    "end_timestamp": end_time,
    "count": 10000
}

response = requests.get(url, params=params)
data = response.json()

print(f"Status Code: {response.status_code}")
print(f"Querying data from: {datetime.fromtimestamp(start_time/1000)} to {datetime.fromtimestamp(end_time/1000)}")

if 'result' in data and 'trades' in data['result']:
    trades = data['result']['trades']
    print(f"Number of trades retrieved: {len(trades)}")
    
    if len(trades) > 0:
        df = pd.DataFrame(trades).sort_values('timestamp').reset_index(drop=True)
        df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')
        print("\nHistorical trades from beginning of perpetual launch:")
        print(df[['datetime', 'price', 'amount', 'direction']][0:500:50])
    else:
        print("No trades found in this time range (BTC-PERPETUAL might not have existed yet)")
else:
    print(json.dumps(data, indent=2))


Status Code: 200
Querying data from: 2018-01-01 00:00:00 to 2025-10-20 00:00:00
Number of trades retrieved: 10000

Historical trades from beginning of perpetual launch:
                   datetime     price    amount direction
0   2025-10-16 23:36:30.390  108172.5    7810.0       buy
50  2025-10-16 23:46:18.778  108160.0   32400.0       buy
100 2025-10-16 23:49:58.092  108085.0    1680.0       buy
150 2025-10-16 23:56:48.900  108152.5     180.0       buy
200 2025-10-17 00:00:15.937  108205.0    9740.0      sell
250 2025-10-17 00:01:58.518  108142.5  250000.0      sell
300 2025-10-17 00:03:03.954  108114.0  314180.0      sell
350 2025-10-17 00:03:14.685  108125.0     100.0      sell
400 2025-10-17 00:04:24.808  108045.0     400.0       buy
450 2025-10-17 00:07:49.125  108085.0     120.0       buy
